In [1]:
import numpy as np
import pickle as pickle
import pandas as pd
import math

from sklearn import svm
from sklearn.preprocessing import normalize

import os
import time

import pandas as pd
import keras.backend as K
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.utils import to_categorical
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import Model
import timeit
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Convolution1D, MaxPooling1D, ZeroPadding1D
from keras.optimizers import SGD
#import cv2, numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
# data_preprocessed_python
os.getcwd()
os.chdir('/content/drive/My Drive')

directory = '/content/drive/MyDrive/sem7_project_phase_1/Fear_level_classification/data_preprocessed_python'

Mounted at /content/drive/


##Get the preprocessed data

In [2]:
channel = [i for i in range(0,32)]
pps = [32,33,34,35,36,37,38,39]

band = [4,8,12,16,25,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz

subjectList = [str(i) for i in range(1, 6)]
subjectList = ['1','2','3','4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28']
# subjectList = ['1','2', '3', '5', '6', '7', '8', '9', '10']
#List of subjects

print("channels:", channel)
print("subject list: ", subjectList)

channels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
subject list:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28']


In [3]:
dir = '/content/drive/MyDrive/sem8/Fear_level_classification/full_channels'
dest = '/content/drive/MyDrive/sem8/Fear_level_classification/full_channels'

In [ ]:

data_training = []
label_training = []
data_testing = []
label_testing = []

for subjects in subjectList:

    with open(dest + "/s" + subjects + '.npy', 'rb') as file:
      sub = np.load(file,allow_pickle=True)
      for i in range (0,sub.shape[0]):
        if i % 5 == 0:
          data_testing.append(sub[i][0])
          # label_testing.append(sub[i][1])
        else:
          data_training.append(sub[i][0])
          # label_training.append(sub[i][1])

for subjects in subjectList:

    with open(dest + "/s_label" + subjects + '.npy', 'rb') as file:
      sub = np.load(file,allow_pickle=True)
      for i in range (0,sub.shape[0]):
        if i % 5 == 0:
          # data_testing.append(sub[i][0])
          label_testing.append(sub[i][0])
        else:
          # data_training.append(sub[i][0])
          label_training.append(sub[i][0])

print("number of subject included: ", len(subjectList))
np.save(dir + '/data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save(dir + '/label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)

np.save(dir + '/data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save(dir + '/label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)

In [4]:
with open(dir+ '/data_training.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)

with open(dir + '/label_training.npy', 'rb') as fileTrainL:
    Y  = np.load(fileTrainL)


print(X.shape)
print(Y.shape)
# X = normalize(X)
# Reshape X to be a 2D array before normalization
X = normalize(X)
Z = np.ravel(Y[:, [1]])

from keras.utils import to_categorical
x_train = np.array(X[:])
y_train = to_categorical(Z)

print(x_train.shape)
print(y_train.shape)

Arousal_Train = np.ravel(Y[:, [0]])
Valence_Train = np.ravel(Y[:, [1]])
Domain_Train = np.ravel(Y[:, [2]])
Like_Train = np.ravel(Y[:, [3]])

(437248, 176)
(437248, 4)
(437248, 176)
(437248, 10)


In [5]:

with open(dir + '/data_testing.npy', 'rb') as fileTrain:
    M  = np.load(fileTrain)

with open(dir + '/label_testing.npy', 'rb') as fileTrainL:
    N  = np.load(fileTrainL)

M = normalize(M)
L = np.ravel(N[:, [1]])

from keras.utils import to_categorical
x_test = np.array(M[:])
y_test = to_categorical(L)
y_test

Arousal_Test = np.ravel(N[:, [0]])
Valence_Test = np.ravel(N[:, [1]])
Domain_Test = np.ravel(N[:, [2]])
Like_Test = np.ravel(N[:, [3]])

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

X_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
X_test = x_test.reshape(x_test.shape[0],x_test.shape[1], 1)
print(x_train.shape)
print(x_test.shape)

from sklearn.model_selection import train_test_split

X_test, X_val, Y_test, Y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Convert lists to numpy arrays if needed
X_test = np.array(X_test)
X_val = np.array(X_val)
Y_test = np.array(Y_test)
Y_val = np.array(Y_val)

print("x_train shape", x_train.shape)
print("X_test shape:", X_test.shape)
print("X_val shape:", X_val.shape)
print("y_train shape", y_train.shape)
print("Y_test shape:", Y_test.shape)
print("Y_val shape:", Y_val.shape)

(437248, 176)
(109312, 176)
x_train shape (437248, 176)
X_test shape: (54656, 176, 1)
X_val shape: (54656, 176, 1)
y_train shape (437248, 10)
Y_test shape: (54656, 10)
Y_val shape: (54656, 10)


##Training the Backbone model - CNN

In [8]:
from keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense
from keras.regularizers import l2

batch_size = 256
num_classes = 10
epochs = 50
input_shape=(x_train.shape[1], 1)
print(input_shape)

(176, 1)


In [ ]:
cnn_model = Sequential()
intput_shape=(x_train.shape[1], 1)
cnn_model.add(Conv1D(128, kernel_size=3,padding = 'same',activation='relu', input_shape=input_shape))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling1D(pool_size=(2)))
cnn_model.add(Conv1D(128,kernel_size=3,padding = 'same', activation='relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling1D(pool_size=(2)))
cnn_model.add(Conv1D(128,kernel_size=3,padding = 'same', activation='relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling1D(pool_size=(2)))
cnn_model.add(Conv1D(128,kernel_size=3,padding = 'same', activation='relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling1D(pool_size=(2)))
cnn_model.add(Conv1D(64,kernel_size=3,padding = 'same', activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=(2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='tanh'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(32, activation='tanh'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(16, activation='relu'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(num_classes, activation='softmax'))
cnn_model.summary()

In [ ]:
cnn_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
cnn_history = cnn_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=100,
          verbose=1,validation_data=(X_val,Y_val))

In [ ]:
cnn_model.save('/content/drive/MyDrive/sem8/Fear_level_classification/models/cnn_model_v1.keras')

In [ ]:
# list all data in history
# print(history.history.keys())
score = cnn_model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

##Knowledge Distillation on the main DEAP dataset with reduced parameters and input features

In [10]:
from keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense
from keras.regularizers import l2

channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31]
batch_size = 256
fear_dir = "/content/drive/MyDrive/sem8/Fear_level_classification/fear_data"


# cnn_model_v1 - trained on 32 eeg and 8 pps
cnn_model = tf.keras.models.load_model('/content/drive/MyDrive/sem8/Fear_level_classification/models/cnn_model_v1.keras')

In [8]:
# Get the last 16 columns - for pps
last_16_columns = np.arange(X_train.shape[1] - 16, X_train.shape[1])
new_channels = []

for i in channel:
  for j in range(5):
    new_channels.append(i*5+j)

print(new_channels)

# Concatenate the specified channels with the last 16 columns
columns_to_extract = np.concatenate((new_channels, last_16_columns))
print(columns_to_extract)

r_x_train = X_train[:, columns_to_extract]
r_x_test = X_test[:, columns_to_extract]
r_x_val = X_val[:, columns_to_extract]
print(r_x_train.shape)
print(r_x_test.shape)
print(r_x_val.shape )

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 30, 31, 32, 33, 34, 55, 56, 57, 58, 59, 65, 66, 67, 68, 69, 85, 86, 87, 88, 89, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 125, 126, 127, 128, 129, 145, 146, 147, 148, 149, 155, 156, 157, 158, 159]
[  5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22
  23  24  30  31  32  33  34  55  56  57  58  59  65  66  67  68  69  85
  86  87  88  89  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 125 126 127 128 129 145 146 147 148 149 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175]
(437248, 86, 1)
(54656, 86, 1)
(54656, 86, 1)


In [11]:
stud_bb_cnn_model = Sequential()
input_shape=(r_x_train.shape[1], 1)
stud_bb_cnn_model.add(Conv1D(128, kernel_size=3,padding = 'same',activation='relu', input_shape=input_shape))
stud_bb_cnn_model.add(BatchNormalization())
stud_bb_cnn_model.add(MaxPooling1D(pool_size=(2)))
stud_bb_cnn_model.add(Conv1D(128,kernel_size=3,padding = 'same', activation='relu'))
stud_bb_cnn_model.add(BatchNormalization())
stud_bb_cnn_model.add(MaxPooling1D(pool_size=(2)))
stud_bb_cnn_model.add(Conv1D(64,kernel_size=3,padding = 'same', activation='relu'))
stud_bb_cnn_model.add(MaxPooling1D(pool_size=(2)))
stud_bb_cnn_model.add(Flatten())
stud_bb_cnn_model.add(Dense(64, activation='tanh'))
stud_bb_cnn_model.add(Dropout(0.2))
stud_bb_cnn_model.add(Dense(16, activation='relu'))
stud_bb_cnn_model.add(Dropout(0.2))
stud_bb_cnn_model.add(Dense(10, activation='softmax'))
stud_bb_cnn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)                    │ (None, 86, 128)             │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 86, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 43, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 43, 128)             │          49,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 43, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 21, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 21, 64)              │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 10, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 640)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          41,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             170 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 117,690 (459.73 KB)

 Trainable params: 117,178 (457.73 KB)

 Non-trainable params: 512 (2.00 KB)

###Trainig the model without Knowledge Distillation

In [16]:
# Recompile the model for fear detection task
stud_bb_cnn_model.compile(loss='categorical_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])

# Train the model with fear data
stud_bb_cnn_history = stud_bb_cnn_model.fit(r_x_train, y_train,
                               batch_size=batch_size,
                               epochs=100,
                               verbose=1,
                               validation_data=(r_x_val, Y_val))

Epoch 1/5


KeyboardInterrupt: 

In [ ]:
stud_bb_cnn_model.save('/content/drive/MyDrive/sem8/Fear_level_classification/models/stud_bb_cnn_model_wkd.keras')

In [17]:
print(stud_bb_cnn_history.history.keys())
score = stud_bb_cnn_model.evaluate(r_x_test, Y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: name 'stud_bb_cnn_history' is not defined

###Training the model with Knowledge Distillation using Soft-Targets

In [13]:
soft_targets = cnn_model.predict(X_train)

# Recompile the model for fear detection task
stud_bb_cnn_model.compile(loss='categorical_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])

# Train the model with fear data
stud_bb_cnn_history = stud_bb_cnn_model.fit(r_x_train, soft_targets,
                               batch_size=batch_size,
                               epochs=100,
                               verbose=1,
                               validation_data=(r_x_val, Y_val))

13664/13664 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step
Epoch 1/100
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.2419 - loss: 1.9712 - val_accuracy: 0.3312 - val_loss: 1.7249
Epoch 2/100
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.3373 - loss: 1.7254 - val_accuracy: 0.4118 - val_loss: 1.5485
Epoch 3/100
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.4016 - loss: 1.5913 - val_accuracy: 0.4515 - val_loss: 1.4520
Epoch 4/100
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.4415 - loss: 1.5089 - val_accuracy: 0.4799 - val_loss: 1.3796
Epoch 5/100
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.4658 - loss: 1.4575 - val_accuracy: 0.5043 - val_loss: 1.3263
Epoch 6/100
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.4827 - loss: 1.4138 - val_accuracy: 0.5108 - val_loss: 1.3120
Epoch 7/100
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.4961 - loss: 1.3861 - val_accuracy: 0.5302 - val_loss: 1.2494
Epoch 8/100
1708/1708 ━━━━━━━━

In [14]:
stud_bb_cnn_model.save('/content/drive/MyDrive/sem8/Fear_level_classification/models/stud_bb_cnn_model_st.keras')

In [15]:
print(stud_bb_cnn_history.history.keys())
score = stud_bb_cnn_model.evaluate(r_x_test,Y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])
1708/1708 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.6575 - loss: 0.9435
Test loss: 0.9413540959358215
Test accuracy: 0.6570733189582825


##Get Fear data
Get feature reduced fear data to fine tune the model

In [16]:
fear_dir = "/content/drive/MyDrive/sem8/Fear_level_classification/fear_data"
# fear_dir = "/content/drive/MyDrive/sem8/Fear_level_classification/fear_data/full_channels"

In [17]:
channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
# channel = [i for i in range(1,33)]
band = [4,8,12,16,25,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['1','2','3','4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28']
# subjectList = [str(i) for i in range(1, 6)]
print(channel)
print(subjectList)
#List of subjects

[1, 2, 3, 4, 6, 11, 13, 17, 19, 20, 21, 25, 29, 31]
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28']


In [ ]:

data_training = []
label_training = []
data_testing = []
label_testing = []

for subjects in subjectList:

    with open(fear_dir + "/s" + subjects + '.npy', 'rb') as file:
      sub = np.load(file,allow_pickle=True)
      for i in range (0,sub.shape[0]):
        if i % 5 == 0:
          data_testing.append(sub[i][0])
          # label_testing.append(sub[i][1])
        else:
          data_training.append(sub[i][0])
          # label_training.append(sub[i][1])

for subjects in subjectList:

    with open(fear_dir + "/s_label" + subjects + '.npy', 'rb') as file:
      sub = np.load(file,allow_pickle=True)
      for i in range (0,sub.shape[0]):
        if i % 5 == 0:
          # data_testing.append(sub[i][0])
          label_testing.append(sub[i][0])
        else:
          # data_training.append(sub[i][0])
          label_training.append(sub[i][0])

np.save(fear_dir + '/r_data_training', np.array(data_training), allow_pickle=True, fix_imports=True)
np.save(fear_dir + '/r_label_training', np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)

np.save(fear_dir + '/r_data_testing', np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save(fear_dir + '/r_label_testing', np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)

In [19]:
# read the independent data
with open(fear_dir + '/r_data_training.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)

with open(fear_dir + '/r_label_training.npy', 'rb') as fileTrainL:
    Y  = np.load(fileTrainL)

print(X.shape)
print(Y.shape)
# X = normalize(X)
# Reshape X to be a 2D array before normalization
X = normalize(X)
from keras.utils import to_categorical
x_train = np.array(X[:])
y_train = np.array(Y[:])
# y_train = to_categorical(Y)

print(X.shape)
print(x_train.shape)
print(y_train.shape)

#read labels
with open(fear_dir + '/data_testing.npy', 'rb') as fileTrain:
    M  = np.load(fileTrain)

with open(fear_dir + '/label_testing.npy', 'rb') as fileTrainL:
    N  = np.load(fileTrainL)

M = normalize(M)
x_test = np.array(M[:])
y_test = np.array(N[:])

print(x_test.shape)
print(y_test.shape)

(48400, 86)
(48400, 4)
(48400, 86)
(48400, 86)
(48400, 4)
(14065, 86)
(14065, 4)


In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
fx_train = scaler.fit_transform(x_train)
fx_test = scaler.fit_transform(x_test)
fx_train = fx_train.reshape(fx_train.shape[0],fx_train.shape[1], 1)
fx_test = fx_test.reshape(fx_test.shape[0],fx_test.shape[1], 1)
print(fx_train.shape)
print(fx_test.shape)
fy_test = y_test
fy_train = y_train

from sklearn.model_selection import train_test_split

fX_test, fX_val, fY_test, fY_val = train_test_split(fx_test, fy_test, test_size=0.5, random_state=42)

# Convert lists to numpy arrays if needed

fX_test = np.array(fX_test)
fX_val = np.array(fX_val)
fY_test = np.array(fY_test)
fY_val = np.array(fY_val)

print("X_train shape", fx_train.shape)
print("X_test shape:", fX_test.shape)
print("X_val shape:", fX_val.shape)
print("Y_train shape", fy_train.shape)
print("Y_test shape:", fY_test.shape)
print("Y_val shape:", fY_val.shape)

(48400, 86, 1)
(14065, 86, 1)
X_train shape (48400, 86, 1)
X_test shape: (7032, 86, 1)
X_val shape: (7033, 86, 1)
Y_train shape (48400, 4)
Y_test shape: (7032, 4)
Y_val shape: (7033, 4)


##Fine Tuning the Knowledge Distilled model for fear data

In [21]:
stud_bb_cnn_model = tf.keras.models.load_model('/content/drive/MyDrive/sem8/Fear_level_classification/models/stud_bb_cnn_model_st.keras')

In [23]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

batch_size = 256
# Assume 'model' is your pre-trained backbone model
input_shape=(fx_train.shape[1], 1)  # Define the input shape

# Create an Input layer with the correct shape
new_input = Input(shape=input_shape)

# Connect the new input to the first layer of your existing model
x = stud_bb_cnn_model.layers[0](new_input)

# Connect the remaining layers of your existing model
for layer in stud_bb_cnn_model.layers[1:-1]:
    x = layer(x)

# Modify the last layer for 4-class fear detection
# model.layers[-1].trainable = False  # Freeze the previous final layer - Not needed as new output is defined
new_output = Dense(4, activation='softmax', name='fear_output')(x)  # New output layer for 4 classes

# Create a new model with the modified output layer
ft_stud_cnn_model = Model(inputs=new_input, outputs=new_output)

# Recompile the model for fear detection task
ft_stud_cnn_model.compile(loss='categorical_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])

# Train the model with fear data
ft_cnn_history = ft_stud_cnn_model.fit(fx_train, fy_train,
                               batch_size=batch_size,
                               epochs=50,
                               verbose=1,
                               validation_data=(fX_val, fY_val))

Epoch 1/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 14s 32ms/step - accuracy: 0.4410 - loss: 2.5613 - val_accuracy: 0.4904 - val_loss: 1.4078
Epoch 2/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.8161 - loss: 0.4997 - val_accuracy: 0.5242 - val_loss: 1.4501
Epoch 3/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8798 - loss: 0.3277 - val_accuracy: 0.4477 - val_loss: 1.8957
Epoch 4/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9171 - loss: 0.2449 - val_accuracy: 0.4261 - val_loss: 2.5107
Epoch 5/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9242 - loss: 0.2210 - val_accuracy: 0.4067 - val_loss: 2.6844
Epoch 6/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9278 - loss: 0.2011 - val_accuracy: 0.4261 - val_loss: 2.6911
Epoch 7/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9417 - loss: 0.1653 - val_accuracy: 0.4337 - val_loss: 2.8464
Epoch 8/50
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9470 - loss: 0.1530 - val_accuracy

In [24]:
#ft_stud_cnn_model_v1 - madel is trained with stud_bb_cnn_model which is trained with soft-targets
ft_stud_cnn_model.save('/content/drive/MyDrive/sem8/Fear_level_classification/models/ft_stud_cnn_model_v1.keras')

In [25]:
ft_stud_cnn_model = tf.keras.models.load_model('/content/drive/MyDrive/sem8/Fear_level_classification/models/ft_stud_cnn_model_v1.keras')

In [26]:
score = ft_stud_cnn_model.evaluate(fX_test, fY_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5559 - loss: 3.0257
Test loss: 3.0219080448150635
Test accuracy: 0.556598424911499
